In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.document_loaders import PyPDFLoader
from operator import itemgetter


import os
import getpass
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    api_key = getpass.getpass("Enter your Google API Key: ")
os.environ["GOOGLE_API_KEY"] = api_key

In [ ]:
session = {}
def getSessionId(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session:
        session[session_id] = ChatMessageHistory()
    return session[session_id]


In [ ]:
loader = PyPDFLoader("llm poisoning.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector_store = InMemoryVectorStore(embedding=embeddings)
vector_store.add_documents(texts[0:50])  # add a smaller batch for testing

retreiver = vector_store.as_retriever(search_kwargs={"k": 3}, search_type="similarity")
retreiver
texts[0:10]

In [ ]:
vector_store.similarity_search("trigger phrase causes a model to comply", k=2)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant. Answer questions based on the provided context.
    
Context: {context}

If you don't know the answer based on the context, say so."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)
    
    # Custom chain with explicit retrieval
custom_chain = (
    {
            "context": itemgetter("input") | retreiver | format_docs,
            "input": itemgetter("input"),
            "chat_history": itemgetter("chat_history")
    }
    | prompt
    | llm
    | StrOutputParser()
)
    
    # Wrap with message history
conversational_chain = RunnableWithMessageHistory(
    custom_chain,
    get_session_history=getSessionId,
    input_messages_key="input",
    history_messages_key="chat_history"
)



response = conversational_chain.invoke(
     {"input" :"What is LLM poisoning?"},
    config={"configurable": {"session_id": "test_session_1"}}
)



In [ ]:
conversational_chain.invoke({"input": "how is llm poisoning affect llms? also my name is Zulfiker. address me with my name"}, {"configurable": {"session_id": "test_session_1"}})

In [ ]:
conversational_chain.invoke({"input": "what was my name"}, {"configurable": {"session_id": "test_session_1"}})